In [ ]:
import gzip
import os
from functools import reduce
from pathlib import Path
import scipy

import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc
from scipy.sparse import csr_matrix
from tqdm.contrib.concurrent import process_map

In [ ]:
pdata = sc.read_h5ad("pseudobulk.h5ad")

In [ ]:
pdata

In [ ]:
# Import DESeq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

In [ ]:
# Infer pathway activities with ulm
tf_acts, tf_pvals = dc.run_ulm(mat=mat, net=collectri)
tf_acts

In [ ]:
pdata.var

In [ ]:
pdata.obs['leiden'] = pdata.obs['leiden'].astype('category')

In [ ]:
sc.tl.rank_genes_groups(pdata, 'leiden', method='wilcoxon')

In [ ]:
pdata.obs.leiden.value_counts()

In [ ]:
gene_list_ranked = sc.get.rank_genes_groups_df(pdata, group=None,key="rank_genes_groups", gene_symbols="gene_name").dropna().head(50)
display(gene_list_ranked[["gene_name", "pvals_adj"]])

In [ ]:
import scanpy_helpers as sh

In [ ]:
ah.plot_dotplot(adata_m)

In [ ]:
path = "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/solo"

In [ ]:
adata_solo = sc.read_h5ad(f"{path}/adata_nodoublet.h5ad")

In [ ]:
adata_solo_int = sc.read_h5ad("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/adata_solo_integrated.h5ad")

In [ ]:
adata_solo_int.var

In [ ]:
# marker genes 
marker_genes= pd.read_csv("/data/scratch/kvalem/projects/2021/microbial-metabolites/tables/40_single-cell-sorted-cd8/marker_genes_cd8t_cells.csv")

In [ ]:
def sentence_case(text):
    return text.capitalize()

In [ ]:
marker_genes['gene'] = marker_genes['gene'].apply(sentence_case)


In [ ]:
marker_genes

In [ ]:
split_columns = marker_genes['cluster'].str.split('_', expand=True)
split_columns.columns = ['CD8', 'cluster', 'cell','genes']
# Concatenate the split columns with the original DataFrame
marker_genes = pd.concat([marker_genes, split_columns], axis=1)

In [ ]:
marker_genes

In [ ]:
marker_genes.rename(columns={"gene":"genesymbol","cell":"cell_type"}, inplace = True)

In [ ]:
marker_genes

In [ ]:
marker_genes = marker_genes.drop_duplicates(subset=['genesymbol'])

In [ ]:
marker_genes.rename(columns={"genesymbol":"gene_identifier"}, inplace = True)

In [ ]:
marker_genes

In [ ]:
marker_genes_ss = marker_genes[['cell_type', 'gene_identifier']]

In [ ]:
marker_genes_ss

In [ ]:
ah = sh.annotation.AnnotationHelper(markers=marker_genes_ss)

In [ ]:
adata_solo.var_names_make_unique()

In [ ]:
acts = sc.read_h5ad("acts.h5ad")

In [ ]:
sc.pp.scale(acts)
acts

In [ ]:
chemokine_receptor_genes = ["CCR4","CCR5","CCR7","CXCR3","CXCR4","CXCR5","CXCR6","CCL3","CCL4","CCL5","CXCL8","XCL1","XCL2","CXCL2","CXCL9","CXCL1","CXCL10"]
chemokine_receptor_genes_sentence_case = [item.capitalize() for item in chemokine_receptor_genes]


In [ ]:
genes_in_adata = adata_solo[:, adata_solo.var.gene_name.isin(chemokine_receptor_genes_sentence_case)].var.gene_name.to_list()

In [ ]:
genes_in_adata

In [ ]:
genes_in_adata

In [ ]:
adata_solo

In [ ]:
def update_columns(row):
   
    if "ICI" not in row['sample_id']:
        row['origin'] = "colon"
    else:
         row['origin'] = "til"
        
    return row

# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
def update_columns(row):
   
    if "ICI1"  in row['sample_id']:
        row['batch_id'] = "ICI1"
    elif "ICI2" in row['sample_id']:
         row['batch_id'] = "ICI2"
    elif row['sample_id'] == "10mix1":
        row['batch_id'] = "1"
    elif row['sample_id'] == "10mix2":
         row['batch_id'] = "2"
        
        
    return row

# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
def update_columns(row):
   
    if row['sample_id'] == "10mix1":
        row['condition'] = "10mix"
    elif row['sample_id'] == "10mix2":
         row['condition'] = "10mix"
    elif row['sample_id'] == "11mix1":
         row['condition'] = "11mix"
    elif row['sample_id'] == "11mix2":
         row['condition'] = "11mix"
    elif row['sample_id'] == "GF1":
         row['condition'] = "GF"
    elif row['sample_id'] == "GF2":
         row['condition'] = "GF"
    elif row['sample_id'] == "GF-ICI2-plus":
         row['condition'] = "GF-plus"
    elif row['sample_id'] == "GF-ICI1-plus":
         row['condition'] = "GF-plus"
    elif row['sample_id'] == "GF-ICI2":
         row['condition'] = "GF"
    elif row['sample_id'] == "GF-ICI1":
         row['condition'] = "GF"
    elif row['sample_id'] == "10mix-ICI1":
         row['condition'] = "10mix"
    elif row['sample_id'] == "10mix-ICI2":
         row['condition'] = "10mix"
    elif row['sample_id'] == "11mix-ICI1":
         row['condition'] = "11mix"
    elif row['sample_id'] == "11mix-ICI2":
         row['condition'] = "11mix"
    
    
        
        
    return row

# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
adata_11mix = adata_solo[adata_solo.obs["sample_id"].isin(["11mix1","11mix2"])]

In [ ]:
adata_10mix = adata_solo[adata_solo.obs["sample_id"].isin(["10mix1","10mix2"])]

In [ ]:
adata_GF = adata_solo[adata_solo.obs["sample_id"].isin(["GF1","GF2"])]

In [ ]:
adata_11mixici = adata_solo[adata_solo.obs["sample_id"].isin(["11mix-ICI1","11mixICI2"])]

In [ ]:
adata_10mixici = adata_solo[adata_solo.obs["sample_id"].isin(["10mix-ICI1","10mix-ICI2"])]

In [ ]:
adata_GFici = adata_solo[adata_solo.obs["sample_id"].isin(["GF-ICI1","GF-ICI2"])]

In [ ]:
adata_GFiciplus = adata_solo[adata_solo.obs["sample_id"].isin(["GF-ICI1-plus","GF-ICI2-plus"])]

In [ ]:
adata_til = adata_solo[adata_solo.obs["origin"].isin(["til"])]

In [ ]:
adata_colon = adata_solo[adata_solo.obs["origin"].isin(["colon"])]

In [ ]:
gene_name = "Cxcr3"
# Check if the gene is present in adata.var
if gene_name in adata_11mix.var_names:
    # Get the index of the gene in adata.var
    gene_index = adata_11mix.var_names.get_loc(gene_name)

    # Filter rows in adata.X based on the presence of the gene
    gene_expression = adata_11mix.X[:, gene_index]

    # Count the number of non-zero entries (presence of the gene)
    num_rows_with_gene = (gene_expression > 0).sum()

    print(f"Number of rows with {gene_name}: {num_rows_with_gene}")
else:
    print(f"{gene_name} not found in adata.var")

In [ ]:
gene_name = "Cxcr3"
# Check if the gene is present in adata.var
if gene_name in adata_10mix.var_names:
    # Get the index of the gene in adata.var
    gene_index = adata_10mix.var_names.get_loc(gene_name)

    # Filter rows in adata.X based on the presence of the gene
    gene_expression = adata_10mix.X[:, gene_index]

    # Count the number of non-zero entries (presence of the gene)
    num_rows_with_gene = (gene_expression > 0).sum()

    print(f"Number of rows with {gene_name}: {num_rows_with_gene}")
else:
    print(f"{gene_name} not found in adata.var")

In [ ]:
gene_name = "Cxcr3"
# Check if the gene is present in adata.var
if gene_name in adata_GF.var_names:
    # Get the index of the gene in adata.var
    gene_index = adata_GF.var_names.get_loc(gene_name)

    # Filter rows in adata.X based on the presence of the gene
    gene_expression = adata_GF.X[:, gene_index]

    # Count the number of non-zero entries (presence of the gene)
    num_rows_with_gene = (gene_expression > 0).sum()

    print(f"Number of rows with {gene_name}: {num_rows_with_gene}")
else:
    print(f"{gene_name} not found in adata.var")

In [ ]:
gene_name = "Cxcl10"
# Check if the gene is present in adata.var
if gene_name in adata_colon.var_names:
    # Get the index of the gene in adata.var
    gene_index = adata_colon.var_names.get_loc(gene_name)

    # Filter rows in adata.X based on the presence of the gene
    gene_expression = adata_colon.X[:, gene_index]

    # Count the number of non-zero entries (presence of the gene)
    num_rows_with_gene = (gene_expression > 0).sum()

    print(f"Number of rows with {gene_name}: {num_rows_with_gene}")
else:
    print(f"{gene_name} not found in adata.var")

In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["10mix1","10mix2","11mix1","11mix2","GF1","GF2"])],color="cell_type",legend_loc="on data", frameon=False)


In [ ]:
if "Cxcr2" in adata_solo.var:
    print("T")

In [ ]:
adata_solo.var.index = adata_solo.var.gene_name

In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["11mix1","11mix2"])], color="Cxcr2", frameon=False)

In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["GF1","GF2"])], color="Cxcr3", frameon=False)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["10mix1","10mix2"])], color="Cxcr3", frameon=False)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["11mix-ICI1","11mix-ICI2"])], color="Cxcr3", frameon=False)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["GF-ICI1","GF-ICI2"])], color="Cxcr3", frameon=False)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["GF-ICI1-plus","GF-ICI2-plus"])], color="Cxcr3", frameon=False)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["10mix-ICI1","10mix-ICI2"])], color="Cxcr3", frameon=False)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["10mix-ICI1","10mix-ICI2","GF-ICI1-plus","GF-ICI2-plus","GF-ICI1","GF-ICI2","11mix-ICI1","11mix-ICI2"])], color="Cxcr3", frameon=False)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["10mix1","10mix2","GF1","GF2","11mix1","11mix2"])], color="Cxcr3", frameon=False)


In [ ]:
if "Cxcl10" in adata_solo_int.var.index:
    print("T")

In [ ]:
adata_solo.obs.columns

In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["cell_type"]], color="Cxcl10", frameon=False)

In [ ]:
"CXCL2","CXCL9","CXCL1","CXCL10"

In [ ]:
genes_in_adata

In [ ]:
adata_solo.var.index = adata_solo.var.ensembl_id

In [ ]:
adata_11mix.var

In [ ]:
adata_solo.var

In [ ]:
adata_solo

In [ ]:
# Subsetting AnnData object
desired_cell_type = "10mix1"
subset_mask = adata_solo.obs["sample_id"] == desired_cell_type
subset_adata = adata_solo[subset_mask, :]

In [ ]:
subset_adata

In [ ]:
list_var_names = adata_solo.raw.var_names.drop_duplicates()

In [ ]:
adata_solo.raw.index= list_var_names

In [ ]:
adata_colon.raw.index= adata_colon.raw.var_names.drop_duplicates()

In [ ]:
adata_solo.raw = None

#del adata.raw

In [ ]:
adata_solo.var.index = adata_solo.var.ensembl_id

In [ ]:
adata_solo

In [ ]:
annotation_dict = {'0': 'Tex',
 '1': 'Tn',
 '2': 'Tisg',
 '3': 'Tsen',
 '4': 'Tstr',
 '5': 'Trm',
 '6': 'Tstr'}

In [ ]:
# Add cell type column based on annotation
adata_solo.obs['cell_type'] = [annotation_dict[clust] for clust in adata_solo.obs['leiden']]

In [ ]:
adata_solo.obs.cell_type

In [ ]:
genes_in_adata

In [ ]:
import scanpy as sc

#genes_in_adata
genes_in_data_list  =  ["ENSMUSG00000045382","ENSMUSG00000026573","ENSMUSG00000034855","ENSMUSG00000047880","ENSMUSG00000047898","ENSMUSG00000048521","ENSMUSG00000079227","ENSMUSG00000035042","ENSMUSG00000000982","ENSMUSG00000018930","ENSMUSG00000037944","ENSMUSG00000050232"]
sc.pl.heatmap(adata_solo, genes_in_adata,gene_symbols="gene_name", groupby='cell_type', swap_axes=True)

In [ ]:
import scanpy as sc

#genes_in_adata
genes_in_data_list  =  ["ENSMUSG00000045382","ENSMUSG00000026573","ENSMUSG00000034855","ENSMUSG00000047880","ENSMUSG00000047898","ENSMUSG00000048521","ENSMUSG00000079227","ENSMUSG00000035042","ENSMUSG00000000982","ENSMUSG00000018930","ENSMUSG00000037944","ENSMUSG00000050232"]
sc.pl.heatmap(adata_solo, genes_in_adata,gene_symbols="gene_name", groupby='origin', swap_axes=True)

In [ ]:
import scanpy as sc

#genes_in_adata
genes_in_data_list  =  ["ENSMUSG00000045382","ENSMUSG00000026573","ENSMUSG00000034855","ENSMUSG00000047880","ENSMUSG00000047898","ENSMUSG00000048521","ENSMUSG00000079227","ENSMUSG00000035042","ENSMUSG00000000982","ENSMUSG00000018930","ENSMUSG00000037944","ENSMUSG00000050232"]
sc.pl.heatmap(adata_solo, genes_in_adata,gene_symbols="gene_name", groupby='condition', swap_axes=True)

In [ ]:
import scanpy as sc

#genes_in_adata
genes_in_data_list  =  ["ENSMUSG00000045382","ENSMUSG00000026573","ENSMUSG00000034855","ENSMUSG00000047880","ENSMUSG00000047898","ENSMUSG00000048521","ENSMUSG00000079227","ENSMUSG00000035042","ENSMUSG00000000982","ENSMUSG00000018930","ENSMUSG00000037944","ENSMUSG00000050232"]
sc.pl.heatmap(adata_solo, genes_in_adata,gene_symbols="gene_name", groupby='sample_id', swap_axes=True)

In [ ]:

sc.pl.dotplot(adata_solo,  genes_in_adata,gene_symbols="gene_name", groupby='cell_type', dendrogram=False, swap_axes = True)

In [ ]:

sc.pl.dotplot(adata_solo,  genes_in_adata,gene_symbols="gene_name", groupby='sample_id',  dendrogram=False, swap_axes = True)

In [ ]:

sc.pl.dotplot(adata_solo,  genes_in_adata,gene_symbols="gene_name", groupby='origin',  dendrogram=False, swap_axes = True)

In [ ]:

sc.pl.dotplot(adata_solo,  genes_in_adata,gene_symbols="gene_name", groupby='condition',  dendrogram=False, swap_axes = True)

In [ ]:
sc.pl.matrixplot(adata_solo,  genes_in_adata,gene_symbols="gene_name", groupby='cell_type', dendrogram=False, swap_axes = True)

In [ ]:
sc.pl.matrixplot(adata_solo,  genes_in_adata,gene_symbols="gene_name", groupby='sample_id',dendrogram=False, swap_axes = True)

In [ ]:
sc.pl.matrixplot(adata_solo,  genes_in_adata,gene_symbols="gene_name", groupby='origin', dendrogram=False, swap_axes = True)

In [ ]:
sc.pl.matrixplot(adata_solo,  genes_in_adata,gene_symbols="gene_name", groupby='condition', dendrogram=False, swap_axes = True)

In [ ]:
gene_name =
gene_index = adata_solo.var.index.get_loc(gene_name)

# Get the corresponding value in the 'enseml_id' column
enseml_id = adata_solo.var.loc[gene_name, 'ensembl_id']

In [ ]:
adata_solo.var.to_csv("adata_solo_var.csv")

In [ ]:
filtered_denoised = sc.read_h5ad("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/filtered/colon_and_tumor_adata_final_filtered.h5ad")

In [ ]:
filtered_denoised.var

In [ ]:
if "Cxcl2" in filtered_denoised.var.gene_name:
    print("T")

In [ ]:
####TAKESHI


In [ ]:
sur_prot = ["Cd69","Cd44","Cd8a","Cd8b1"]

In [ ]:

sc.pl.dotplot(adata_solo,  sur_prot,gene_symbols="gene_name", groupby='cell_type',  dendrogram=False, swap_axes = True)

In [ ]:

sc.pl.dotplot(adata_solo,  sur_prot,gene_symbols="gene_name", groupby='sample_id', dendrogram=False, swap_axes = True)

In [ ]:

sc.pl.dotplot(adata_solo,  sur_prot,gene_symbols="gene_name", groupby='condition', dendrogram=False, swap_axes = True)

In [ ]:

sc.pl.dotplot(adata_solo,  sur_prot,gene_symbols="gene_name", groupby='origin', dendrogram=False, swap_axes = True)

In [ ]:
genes_heatmap = ["Klrk1", "Ccl4", "Klrc1", "Gzma", "Dapk2", "Gzmk", "Gpr55","Ifng","Mcm5","Ifit3","Ramp3", "Gzmb",]

In [ ]:
import scanpy as sc

sc.pl.heatmap(adata_solo, genes_heatmap,gene_symbols="gene_name", groupby='cell_type', swap_axes=True)

In [ ]:
import scanpy as sc

sc.pl.heatmap(adata_solo, genes_heatmap,gene_symbols="gene_name", groupby='origin', swap_axes=True)

In [ ]:
import scanpy as sc

sc.pl.heatmap(adata_solo, genes_heatmap,gene_symbols="gene_name", groupby='condition', swap_axes=True)

In [ ]:
import scanpy as sc

sc.pl.heatmap(adata_solo, genes_heatmap,gene_symbols="gene_name", groupby='sample_id', swap_axes=True)

In [ ]:
adata_solo.var.index = adata_solo.var.ensembl_id

In [ ]:

sc.pl.dotplot(adata_solo,  genes_heatmap,gene_symbols="gene_name", groupby='cell_type',  dendrogram=False, swap_axes = True)

In [ ]:

sc.pl.dotplot(adata_solo,  genes_heatmap,gene_symbols="gene_name", groupby='sample_id', dendrogram=False, swap_axes = True)

In [ ]:
sc.pl.dotplot(adata_solo,  genes_heatmap,gene_symbols="gene_name", groupby='origin', dendrogram=False, swap_axes = True)

In [ ]:
sc.pl.dotplot(adata_solo,  genes_heatmap,gene_symbols="gene_name", groupby='condition', dendrogram=False, swap_axes = True)

In [ ]:
genes_heatmap_cytokine = ["Csf1", "Il10ra","Il16", "Il18rap","Il21r","Il2rb","Il2rg","Il9r","Adam10","Adam8","Cd70"]

In [ ]:
genes_heatmap_cytotoxicty = ["Ifng", "Tnf","Cxcl9"]

In [ ]:
adata_solo

In [ ]:
sc.pl.dotplot(adata_solo, genes_heatmap_cytokine,gene_symbols="gene_name", groupby='sample_id',layer= "logcounts", dendrogram=False, swap_axes = True)

In [ ]:
sc.pl.dotplot(adata_solo,  genes_heatmap_cytokine,gene_symbols="gene_name", groupby='cell_type', dendrogram=False, swap_axes = True)

In [ ]:
sc.pl.dotplot(adata_solo,  genes_heatmap_cytokine,gene_symbols="gene_name", groupby='origin', dendrogram=False, swap_axes = True)

In [ ]:
sc.pl.dotplot(adata_solo,  genes_heatmap_cytokine,gene_symbols="gene_name", groupby='condition', dendrogram=False, swap_axes = True)

In [ ]:
import pandas as pd

# Assuming you have a list of sample IDs and a corresponding list of cell counts
sample_ids = ["10mix", "11mix", "GF"]  # Replace with your actual sample IDs
number_of_cells = [826, 1111, 175]  # Replace with your actual cell count data

# Create a dictionary to store the data
data = {
    "sample_id": sample_ids,
    "number_of_cells": number_of_cells
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

print(df)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df is your DataFrame with sample IDs and cell counts

# Create a horizontal bar plot using matplotlib
plt.figure(figsize=(5, 3))  # Set the figure size

# Plot the data as horizontal bars
plt.barh(df['sample_id'], df['number_of_cells'],  fill=False)

# Customize the plot
plt.xlabel('Number of Cells')
plt.ylabel('Sample')
plt.title('')
plt.tight_layout()  # Adjust layout for better spacing

# Show the plot
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming cell_type_df is your DataFrame with cell type counts

# Create a bar plot using matplotlib
plt.figure(figsize=(3, 5))  # Set the figure size

# Plot the data
plt.bar(df['sample_id'], df['number_of_cells'], fill=False)

# Customize the plot
plt.xlabel('Sample')
plt.ylabel('N. of cells')
plt.title('Sample Counts')
plt.xticks(rotation=90)  # Rotate x-axis labels for better visibility


# Show the plot
plt.tight_layout()  # Adjust layout for better spacing
plt.show()
